# **Chloroplast Curation**

The purpose of this notebook is to describe the identification, assessment and curation of abnormal cds found after automated annotation (using [GeSeq](https://chlorobox.mpimp-golm.mpg.de/geseq.html)) of the _Plukenetia volubilis_ chloroplast sequence.

These analysis were carried out using python and specifically [Biopython](https://biopython.org/), starting from the genbank file `chloroplast_annotated_raw.gb` generated by GeSeq, and ended up in a modified genbank file `chloroplast_annotated_curated.gb`

***

First, some necesary modules are loaded.

In [1]:
from Bio import Entrez
from Bio import SeqIO
from Bio import Align
from Bio.Align import substitution_matrices
from Bio.Data import CodonTable
from Bio.SeqFeature import FeatureLocation, CompoundLocation
from Bio.Seq import Seq

/home/sivico26/miniconda3/envs/Sci/lib/python3.7/site-packages/Bio/Align/substitution_matrices/__init__.py:21: BiopythonExperimentalWarning: Bio.Align.substitution_matrices is an experimental module which may still undergo significant changes. In particular, the location of this module may change, and the Array class defined in this module may be moved to other existing or new modules in Biopython.
  BiopythonExperimentalWarning)


And the "raw" genbank file is loaded too:

In [2]:
sacha = SeqIO.read("chloroplast_annotated_raw.gb","gb")

Identification of abnormal cds was executed based on the following criteria:

- The sequence must start with a start codon.
- The sequence has to have one, and just one, stop codon. Additionally it must be located at the end of the sequence.
- The sequence has to be exactly divisible in codons (_i.e._ there should not be remaining nucleotides outside codons).

The following functions check the aforementioned conditions and extracts the cds that do not fulfill them.

In [3]:
def check_start(cds, gen_cod):
    """Return True if the protein starts by a start codon (according to the genetic code)
    returns False otherwise"""
    starts = CodonTable.unambiguous_dna_by_id[gen_cod].start_codons
    return True if cds.seq[:3] in starts else False

def check_exact_cod(cds):
    "Calculate the number of nucleotides outside codons for a protein"
    return len(cds.seq) % 3

def check_stops(cds, gen_cod):
    """Returns the number of stop codons of a protein. 
    If there is exactly one, instead return True if it is 
    at the end of que sequence and false otherwise."""
    
    prot = cds.seq.translate(table = gen_cod)
    stops = prot.count("*")
    if stops != 1:
        return stops
    else:
        return True if prot[-1] == "*" else False

def make_table(bad_cds):
    print("index\tcds\tstart\tcod_excess\tstop")
    for cds, reasons in bad_cds.items():
        aux_list = [str(reasons["index"]),cds]
        aux_list.append(str(reasons["start"]))
        aux_list.append(str(reasons["cod"]))
        aux_list.append(str(reasons["end"]))
        print("\t".join(aux_list))
    
def check_cds(req, gen_cod = 11):
    anomalus = {}
    for i,ft in enumerate(req.features):
        if ft.type == "CDS":
            reasons = {}
            cds = ft.extract(req)
            reasons["start"] = check_start(cds, gen_cod)
            reasons["cod"] = check_exact_cod(cds)
            reasons["end"] = check_stops(cds, gen_cod)
            
            if reasons["start"] != True or reasons["cod"] != 0 or reasons["end"] != True:
                reasons["index"] = i
                anomalus[ft.qualifiers["gene"][0]] = reasons
    make_table(anomalus)
    return anomalus

In [4]:
anomalus = check_cds(sacha)

index	cds	start	cod_excess	stop
62	rpoC1	True	1	0
75	atpF	True	2	13
99	ndhK	True	2	0
168	petB	False	1	13
170	petD	False	2	18
178	infA	False	0	2
184	rpl16	False	1	10
188	rpl22	True	1	5
243	ycf1	True	1	False
254	ndhI	True	0	0
256	ndhG	True	0	2
262	ndhD	False	0	True
264	ccsA	True	2	0
270	ndhF	True	1	0
272	ycf1-fragment	True	2	0


/home/sivico26/miniconda3/envs/Sci/lib/python3.7/site-packages/Bio/Seq.py:2859: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


Thus, 15 cds were retrieved, from which at least 14 had to be reviewed (ycf1-fragment is naturally fragmented by the IRs). Separated by kind of anomaly:

In [5]:
anom_start_ft = set()
anom_cod_ft = set()
anom_end_ft = set()
for cds, carac in anomalus.items():
    if not carac["start"]:
        anom_start_ft.add(cds)
    if carac["cod"] != 0:
        anom_cod_ft.add(cds)
    if carac["end"] is not True:
        anom_end_ft.add(cds)

In [6]:
print(len(anom_start_ft))
print(len(anom_cod_ft))
print(len(anom_end_ft))

5
11
14


In summary (excluding ycf1-fragment):

- Proteins without start codon: 5
- Proteins with nucleotides outside its codons: 10
- Proteins with abnormal Stops: 13

In [7]:
anom_ft = [sacha.features[key['index']] for key in anomalus.values()]

***

The starting codon anomalies were addressed first

### **Starting codon**

In [8]:
anom_start_ft

{'infA', 'ndhD', 'petB', 'petD', 'rpl16'}

In [9]:
for ft in anom_ft:
    name = ft.qualifiers["gene"][0]
    if name in anom_start_ft:
        idx = int(anomalus[name]["index"])
        start = str(ft.extract(sacha).seq[:3])
        if start in CodonTable.unambiguous_dna_by_id[11].start_codons:
            print(f'{start} from {name} is a start codon')
        else:
            print(sacha.features[idx - 1].qualifiers["note"][0])

blatX_hit petB_Rcommunis_GeSeq-SRS_v6, position 3 - 648, psl score 98.3, coverage 99.69%, match 97.99%
blatX_hit petD_Hbrasiliensis_GeSeq-SRS_v6, position 8 - 504, psl score 97.4, coverage 98.61%, match 96.03%
blatX_hit infA_Vvinifera_GeSeq-SRS_v6, position 57 - 122, psl score 80.4, coverage 28.21%, match 22.65%
blatX_hit rpl16_Hbrasiliensis_GeSeq-SRS_v6, position 6 - 408, psl score 96.8, coverage 98.77%, match 95.59%
blatX_hit ndhD_Rcommunis_GeSeq-SRS_v6, position 1 - 1503, psl score 96.8, coverage 100.00%, match 96.74%


Something that all these genes have in common is that they were annotated __by reference__ using __blatX__. Also, all except ndhD, did not start in the first nucleotide of their reference, which might explain why they were missing their start codons.

Also the match with infA has less than a third of the protein coverage, so it is unlikely that this protein is functional.

### __Finding the missing start codons__

If the cds, even though partial, were real, the starting codons would not be far from annotated. Thus, the starting codons were searched upstream and downstream

In [10]:
def offset_loc(location, offset):
    """Given a locaton in a feature location, return an offseted location by 'offset' bp (integer number)
    Here, a positive value of the offset would return a a location moved FORWARD, while a negative
    value would return a location moved BACKWARD"""
    if offset == 0:
        raise ValueError(f"Expected an offset different from 0")
    elif isinstance(location, FeatureLocation):
        return location + location.strand * offset
    elif isinstance(location, CompoundLocation):
        return sum(part + part.strand * offset for part in location.parts)
    else:
        raise TypeError(f"Expected either FeatureLocation or CompoundLocation object ({type(location)} given)")

In [11]:
def find_start(feature, mother, starts, forward = False, start_offset = 1):
    """Given a feature(cds) and its mother sequence search for a codon, present 
    in starts. Start_offset modifies the location where of the feature before the search.
    If forward is True the search is made downstream the beginning of the sequence, 
    otherwise it is made upstream. 
    
    Returns how many positions, either upstream or downstream the starting codon was found.
    """
    start = feature.extract(mother).seq[:3]
    name = feature.qualifiers["gene"][0]
    if start in starts:
        print(f"The start {start} of {name} is a start codon")
    elif forward:
        i = start_offset if start_offset > 0 else start_offset * -1
        missing_start = True
        while missing_start:
            start = offset_loc(feature.location, i).extract(mother).seq[:3]
            if start in starts:
                print(f"A start codon {start} was found {i} bp downstream the sequence for {name}")
                missing_start = False
                return i 
            i += 1
    else:
        i = start_offset if start_offset < 0 else start_offset * -1
        missing_start = True
        while missing_start:
            start = offset_loc(feature.location, i).extract(mother).seq[:3]
            if start in starts:
                print(f"A start codon {start} was found {i} bp upstream the sequence for {name}")
                missing_start = False
                return i
            i -= 1

Upstream:

In [12]:
for ft in anom_ft:
    name = ft.qualifiers["gene"][0]
    if name in anom_start_ft:
        find_start(ft, sacha, CodonTable.unambiguous_dna_by_id[11].start_codons)

A start codon ATC was found -4 bp upstream the sequence for petB
A start codon ATA was found -1 bp upstream the sequence for petD
A start codon ATG was found -7 bp upstream the sequence for infA
A start codon ATA was found -1 bp upstream the sequence for rpl16
A start codon TTG was found -11 bp upstream the sequence for ndhD


Downstream:

In [13]:
for ft in anom_ft:
    name = ft.qualifiers["gene"][0]
    if name in anom_start_ft:
        find_start(ft, sacha, CodonTable.unambiguous_dna_by_id[11].start_codons, forward = True)

A start codon ATA was found 2 bp downstream the sequence for petB
A start codon CTG was found 12 bp downstream the sequence for petD
A start codon ATG was found 1 bp downstream the sequence for infA
A start codon ATT was found 18 bp downstream the sequence for rpl16
A start codon ATT was found 4 bp downstream the sequence for ndhD


As suspected, starting codons were not far either upstream or downstream. 

The ndhD was quite particular, as in three other cases (_[Jatroha curcas](https://www.ncbi.nlm.nih.gov/nucleotide/NC_012224.1), [Nicotiana tabacum](https://www.ncbi.nlm.nih.gov/nucleotide/NC_001879.2) and [Arabidopsis thaliana](https://www.ncbi.nlm.nih.gov/nucleotide/NC_000932.1)_) it starts by ACG, even though ACG is not a start codon for the bacterial genetic code. 

#### __Finding a reasonable start__

Now that possible starts for protein were located, the next step is to assess if taking those gives a reasonable protein. 

One way to do so is to compare with a trustful reference, and align the modified proteins with their respective homologues. Usually, the closer the reference the better. However, as _Ricinus communis_ chloroplast had presented issues before, for these analysis the model organism _[Arabidopsis thaliana](https://www.ncbi.nlm.nih.gov/nucleotide/NC_000932.1)_ was preferred.

In [14]:
Entrez.email = "svillanu@eafit.edu.co"

handle = Entrez.efetch(db="nucleotide", id="NC_000932", rettype="gb", retmode="text")
arabidopsis = SeqIO.read(handle, "gb")

In [15]:
def find_protein(feature, mother, ref, starts, gen_cod, reach = 50):
    """Take a feature with its mother sequence and look for alternatives 
    based in the starts given, within a reach upstream and downstream. 
    
    Returns a dictionary where the keys are the offset relative to the start
    of the protein and the values are the alignment of the modified protein 
    with the homologue of the reference"""
    
    name = feature.qualifiers["gene"][0]
    aligner = Align.PairwiseAligner()
    aligner.open_gap_score = -5 
    aligner.extend_gap_score = -0.5
    aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
    ## To Do: are these good alignment defaults?
    
    
    found = False
    for ft in ref.features:
        if ft.type == "CDS" and ft.qualifiers["gene"][0] == name:
            target = ft.extract(ref).seq.translate(table = gen_cod)
            found = True
    
    if not found:
        raise NameError(f"Protein {name} could not be found in the provided reference") 
    
    alns = {}
    max_score = -1000 ## To Do: look for a better default
    idx_max = 0 
    i = 1
    while abs(i) < reach:
        i = find_start(feature, mother, starts, forward=True, start_offset=i)
        query = offset_loc(feature.location, i).extract(mother).seq.translate(table = gen_cod)
        alns[i] = aligner.align(target, query)
        if alns[i].score > max_score:
            max_score = alns[i].score
            idx_max = i
        i += 1
        
    i = -1
    while abs(i) < reach:
        i = find_start(feature, mother, starts, forward=False, start_offset=i)
        query = offset_loc(feature.location, i).extract(mother).seq.translate(table = gen_cod)
        alns[i] = aligner.align(target, query)
        if alns[i].score > max_score:
            max_score = alns[i].score
            idx_max = i
        i -= 1
    print(f"A total of {len(alns.keys())} options were found in the range of {reach}")
    print(f"The best option found in the explored range was found with an offset of {idx_max} with an score of {max_score}")
    return alns

petB was taken as an example of the above function. 

In [16]:
idx = anomalus["petB"]["index"]
alns = find_protein(sacha.features[idx], sacha, arabidopsis, CodonTable.unambiguous_dna_by_id[11].start_codons, 11)

A start codon ATA was found 2 bp downstream the sequence for petB
A start codon ATG was found 11 bp downstream the sequence for petB
A start codon ATT was found 14 bp downstream the sequence for petB
A start codon TTG was found 15 bp downstream the sequence for petB
A start codon ATT was found 37 bp downstream the sequence for petB
A start codon ATT was found 46 bp downstream the sequence for petB
A start codon TTG was found 47 bp downstream the sequence for petB
A start codon ATG was found 53 bp downstream the sequence for petB
A start codon ATC was found -4 bp upstream the sequence for petB
A start codon CTG was found -15 bp upstream the sequence for petB
A start codon ATA was found -41 bp upstream the sequence for petB
A start codon ATA was found -43 bp upstream the sequence for petB
A start codon ATT was found -49 bp upstream the sequence for petB
A total of 13 options were found in the range of 50
The best option found in the explored range was found with an offset of -41 with an 

Looking at the score of these alignments:

In [17]:
[(i,aln.score) for i,aln in alns.items()]

[(2, 57.5),
 (11, 61.5),
 (14, 57.5),
 (15, 81.0),
 (37, 1014.5),
 (46, 998.5),
 (47, 40.5),
 (53, 40.0),
 (-4, 64.5),
 (-15, 78.5),
 (-41, 1020.5),
 (-43, 62.5),
 (-49, 67.5)]

The first number of the tuple is the offset where the start codon was found (a positive value implies it was found downstream, while a negative value means it was found upstream).The second number of the tuple is the score of the alignment.

Based on the scores, 37 and 46 downstream and 41 upstream were reasonalbe candidates

#### **Finding stop codons**

Now that the start candidates are set, the stop codons have to be located if missing.

In [18]:
def find_stop(feature, mother, gen_cod, offset = 0):
    """Given a feature and its mother sequence search for 
    the next stop codon if there is none. Also the nucleotides that
    do not belong to any codon are trimmed from the end of the sequence.
    
    If succesful, it returns a modified location that fulfills cds requiriments"""
    # To do: modify to use with CompoundLocations with offset 
    if offset != 0:
        loc = feature.location + offset * feature.location.strand ## This only applies to FeatureLocations ::: To Do: generalize to CompoundLocation
    else:
        loc = feature.location
    prot = loc.extract(mother).seq.translate(table=gen_cod)
    stops = prot.count("*")
    if stops != 0:
        print(f"This feature already has {stops} stop codons")
        if stops == 1 and prot[-1] == "*":
            print(f"This protein has already a unique stop codon in the end of the sequence")
        elif stops == 1:
            print(f"And it is located at position {prot.find('*')} of {len(prot)}")
        else:
            print(f"And the first of them is located at position {prot.find('*')} of {len(prot)}")
    else:
        # Going downstream
        name = feature.qualifiers["gene"][0]
        not_found = True
        i = 0
        excess = len(loc) % 3 
        if isinstance(loc, FeatureLocation):
            if excess != 0:
                if loc.strand == 1:
                    loc = FeatureLocation(loc.start, loc.end - excess*loc.strand, loc.strand)
                elif loc.strand == -1:
                    loc = FeatureLocation(loc.start - excess*loc.strand, loc.end, loc.strand)
            while not_found:
                i += 1
                if loc.strand == 1:
                    loc = FeatureLocation(loc.start, loc.end + 3*loc.strand, loc.strand)
                elif loc.strand == -1:
                    loc = FeatureLocation(loc.start + 3*loc.strand, loc.end, loc.strand)
                    
                if loc.extract(mother).seq.translate(table=gen_cod).endswith("*"):
                    print(f"A stop for {name} was found {i} codons downstream ({i*3} bp) and the protein length would be {int(len(loc)/3)} aa")
                    not_found = False
        elif isinstance(loc, CompoundLocation): # To Do: modify it to work with (-) locations (and heterogeneous?)
            end_exon = loc.parts[-1] 
            if excess != 0:
                if end_exon.strand == 1:
                    end_exon = FeatureLocation(end_exon.start, end_exon.end - excess*end_exon.strand, end_exon.strand)
                elif end_exon.strand == -1:
                    end_exon = FeatureLocation(end_exon.start - excess*end_exon.strand, end_exon.end, end_exon.strand)
            while not_found:
                i += 1
                end_exon = FeatureLocation(end_exon.start, end_exon.end + 3*loc.strand, end_exon.strand)
                loc = sum(part for part in loc.parts[:-1]) + end_exon
                if loc.extract(mother).seq.translate(table=gen_cod)[-1] == "*":
                    print(f"A stop for {name} was found {i} codons downstream ({i*3} bp) and the protein length would be {int(len(loc)/3)} aa")
                    not_found = False
        else:
            raise TypeError(f"Expected either FeatureLocation or CompoundLocation object ({type(location)} given)")
        return loc

Continuing the petB example

In [19]:
idx = anomalus["petB"]["index"]
new_petb = find_stop(sacha.features[idx], sacha, 11, -41)

A stop for petB was found 14 codons downstream (42 bp) and the protein length would be 229 aa


In [20]:
new_petb.extract(sacha).seq.translate(table=11, cds=True)

Seq('MRFSEGEFLWFTYLNKVYDWFEERLEIQAIADDITSKYVPPHVNIFYCLGGITL...GPL', ExtendedIUPACProtein())

The above protein fulfilled the cds characteristics, but did it have biological sense? An alignment to the reference shed some light.

In [21]:
aligner = Align.PairwiseAligner()
aligner.open_gap_score = -5 
aligner.extend_gap_score = -0.5
aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")

for ft in arabidopsis.features:
    if ft.type == "CDS" and ft.qualifiers["gene"][0] == "petB":
        target = ft.extract(arabidopsis).seq.translate(table = 11)

query = new_petb.extract(sacha).seq.translate(table=11, cds=True)
print(aligner.align(target, query)[0])

M-------------SKVYDWFEERLEIQAIADDITSKYVPPHVNIFYCLGGITLTCFLVQVATGFAMTFYYRPTVTEAFASVQYIMTEANFGWLIRSVHRWSASMMVLMMILHVFRVYLTGGFKKPRELTWVTGVVLGVLTASFGVTGYSLPWDQIGYWAVKIVTGVPDAIPVIGSPLVELLRGSASVGQSTLTRFYSLHTFVLPLLTAVFMLMHFLMIRKQGISGPL*
|-------------.||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||.||||||||||||||||||||||||||||||.|||||||||||||||||||||||||||||||||||||||||||||||.|||||||||||-
MRFSEGEFLWFTYLNKVYDWFEERLEIQAIADDITSKYVPPHVNIFYCLGGITLTCFLVQVATGFAMTFYYRPTVTEAFASVQYIMTEANFGWLIRSVHRWSASMMVLMMILHVFRVYLTGGFKKPRELTWVTGVVLSVLTASFGVTGYSLPWDQIGYWAVKIVTGVPEAIPVIGSPLVELLRGSASVGQSTLTRFYSLHTFVLPLLTAVFMLMHFPMIRKQGISGPL-



It seems as the best cds achievable for this gene. But does that insertion goes there? further assessment should be done to be more sure. If it is compared with _[Ricinus communis](https://www.ncbi.nlm.nih.gov/nucleotide/NC_016736.1)_ as reference:

In [22]:
handle = Entrez.efetch(db="nucleotide", id="NC_016736", rettype="gb", retmode="text")
ricinus = SeqIO.read(handle, "gb")

In [23]:
aligner = Align.PairwiseAligner()
aligner.open_gap_score = -5 
aligner.extend_gap_score = -0.5
aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")

for ft in ricinus.features:
    if ft.type == "CDS" and ft.qualifiers["gene"][0] == "petB":
        target = ft.extract(ricinus).seq.translate(table = 11, cds=True)

query = new_petb.extract(sacha).seq.translate(table=11, cds=True)
print(aligner.align(target, query)[0])

MRFSEGEALWFTYLNKVYDWFEERLEIQAIADDITSKYVPPHVNIFYCLGGITLTCFLVQVATGFAMTFYYRPTVTEAFASVQYIMTEANFGWLIRSVHRWSASMMVLMMILHVFRVYLTGGFKKPRELTWVTGVVLAVLTASFGVTGYSLPWDQIGYWAVKIVTGVPEAIPVIGSPLVELLRGSASVGQSTLTRFYSLHTFVLPLLTAVFMLMHFPMIRKQGISGPL
|||||||.|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||.||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
MRFSEGEFLWFTYLNKVYDWFEERLEIQAIADDITSKYVPPHVNIFYCLGGITLTCFLVQVATGFAMTFYYRPTVTEAFASVQYIMTEANFGWLIRSVHRWSASMMVLMMILHVFRVYLTGGFKKPRELTWVTGVVLSVLTASFGVTGYSLPWDQIGYWAVKIVTGVPEAIPVIGSPLVELLRGSASVGQSTLTRFYSLHTFVLPLLTAVFMLMHFPMIRKQGISGPL



As the insertion is shared with _Ricinus communis_ it might be considered natural.

***

### __Fixing the remaining starting proteins__

Again the cds with starts anomalies are:

In [24]:
anom_start_ft

{'infA', 'ndhD', 'petB', 'petD', 'rpl16'}

As aforementioned, infA can be discarded because its coverage is too low to be functional. Additionally, besides the difference in the start codon, ndhD has no cds problems.

In [25]:
aligner = Align.PairwiseAligner()
aligner.open_gap_score = -5 
aligner.extend_gap_score = -0.5
aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")

for ft in arabidopsis.features:
    if ft.type == "CDS" and ft.qualifiers["gene"][0] == "ndhD":
        target = ft.extract(arabidopsis).seq.translate(table = 11)

query = sacha.features[anomalus["ndhD"]["index"]].extract(sacha).seq.translate(table=11)
print(aligner.align(target, query)[0])

TNDFPWLTIIVVFPISAGSLMLFLPHRGNKVNKWYTICICILELLLTTYAFCYNFKMDDPLIQLSEDYKWIDFFDFYWRMGIDGLSIGTILLTGFITTLATLAAFPVTRDSRFFHFLMLAMYSGQIGSFSSRDLLLFFIMWELELIPVYLLLSMWGGKKRLYSATKFILYTAGSSIFLLIGVLGISLYGSNEPTLNLELLANKSYPVTLEILFYIGFLIAFAVKSPIIPLHTWLPDTHGEAHYSTCMLLAGILLKMGAYGLVRINMELLPHAHSMFSPWLLVVGTIQIIYAASTSPGQRNLKKRIAYSSVSHMGFIIIGISSITDPGLNGAILQIISHGFIGAALFFLAGTSYDRIRLVYLDEMGGMAISIPKIFTMFTILSMASLALPGMSGFIAEFIVFFGIITSQKYFLISKIFIIFVMAIGMILTPIYLLSMLRQMFYGYKLINIKNFSFFDSGPRELFLSISILLPIIGIGIYPDFVLSLASDKVESILSNYFYG*
||.||||||.||.|.|||||..|.||.||||..|||..|||||.||.||.|.|.|..|||||||.||||||.|||||||.||||.|...|||||||||||||||.|.|||||.||||||||||||||.|||.|||||||||||||||||||||.|||||||||||||||||||.|.|||.|.|||.|||||||....|..||.||||.|||.|||||||||||||||||||.||||||||||||||||||||||||||||||||||||||||||.|||||...|.|||||||||||||||||.|||||||||||||||||.||.|.|.|||||||||||||||||||||||||||||.||||||||||...|||||.|.|||.|||||||||||.||...||||||.|||.|.|||.|.|.||.||||||||||||||||||||||....|..||||||||||.|||||||.|||||||||..||..|.||..|||||

Hence both were excluded

In [26]:
anom_start_ft_rem = anom_start_ft.copy()
for prot in ["ndhD","infA"]:
    anom_start_ft_rem.discard(prot)

In [27]:
anom_start_ft_rem

{'petB', 'petD', 'rpl16'}

If the above process with petB is replicated for the other start anomalous cds the result is the following:

In [28]:
alt_cand = {}
alt_loc = {}
starts = CodonTable.unambiguous_dna_by_id[11].start_codons
for ft in anom_ft:
    name = ft.qualifiers["gene"][0]
    if name in anom_start_ft_rem:
        print(f"Beginnig with {name}")
        alns = find_protein(ft, sacha, arabidopsis, starts, 11, reach = 50)
        dict_scores = {i:aln.score for i,aln in alns.items()}
        avg_score = sum(dict_scores.values()) / len(dict_scores)
        aln_candidates = {i:aln[0] for i, aln in alns.items() if aln.score > avg_score} ## The filtering criterion could be improved
        print(f"A total of {len(aln_candidates)} candidates were found for {name}")
        alt_cand[name] = aln_candidates
        
        alt_loc[name] = {}
        for i, aln in aln_candidates.items():
            alt_loc[name][i] = find_stop(ft, sacha, 11, offset = i)

Beginnig with petB
A start codon ATA was found 2 bp downstream the sequence for petB
A start codon ATG was found 11 bp downstream the sequence for petB
A start codon ATT was found 14 bp downstream the sequence for petB
A start codon TTG was found 15 bp downstream the sequence for petB
A start codon ATT was found 37 bp downstream the sequence for petB
A start codon ATT was found 46 bp downstream the sequence for petB
A start codon TTG was found 47 bp downstream the sequence for petB
A start codon ATG was found 53 bp downstream the sequence for petB
A start codon ATC was found -4 bp upstream the sequence for petB
A start codon CTG was found -15 bp upstream the sequence for petB
A start codon ATA was found -41 bp upstream the sequence for petB
A start codon ATA was found -43 bp upstream the sequence for petB
A start codon ATT was found -49 bp upstream the sequence for petB
A total of 13 options were found in the range of 50
The best option found in the explored range was found with an off

In [29]:
alt_loc

{'petB': {37: None,
  46: None,
  -41: FeatureLocation(ExactPosition(81013), ExactPosition(81700), strand=1)},
 'petD': {17: None,
  44: None,
  56: None,
  -1: None,
  -7: None,
  -25: FeatureLocation(ExactPosition(82687), ExactPosition(83188), strand=1),
  -49: FeatureLocation(ExactPosition(82663), ExactPosition(83188), strand=1)},
 'rpl16': {43: None,
  52: None,
  -8: FeatureLocation(ExactPosition(86871), ExactPosition(87282), strand=-1),
  -38: None}}

As a result, 4 alternatives were obtained for 3 proteins (petD has 2). 
A final alignment with the reference to check the results:

#### __petB__

In [30]:
name = "petB"
off = -41
ref = arabidopsis
for ft in ref.features:
    if ft.type == "CDS":
        if ft.qualifiers["gene"][0] == name:
            target = ft.extract(arabidopsis).seq.translate(table=11, cds=True)

query = alt_loc[name][off].extract(sacha).seq.translate(table=11, cds=True)
alns = aligner.align(target, query)

In [31]:
print(alns[0])

M-------------SKVYDWFEERLEIQAIADDITSKYVPPHVNIFYCLGGITLTCFLVQVATGFAMTFYYRPTVTEAFASVQYIMTEANFGWLIRSVHRWSASMMVLMMILHVFRVYLTGGFKKPRELTWVTGVVLGVLTASFGVTGYSLPWDQIGYWAVKIVTGVPDAIPVIGSPLVELLRGSASVGQSTLTRFYSLHTFVLPLLTAVFMLMHFLMIRKQGISGPL
|-------------.||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||.||||||||||||||||||||||||||||||.|||||||||||||||||||||||||||||||||||||||||||||||.|||||||||||
MRFSEGEFLWFTYLNKVYDWFEERLEIQAIADDITSKYVPPHVNIFYCLGGITLTCFLVQVATGFAMTFYYRPTVTEAFASVQYIMTEANFGWLIRSVHRWSASMMVLMMILHVFRVYLTGGFKKPRELTWVTGVVLSVLTASFGVTGYSLPWDQIGYWAVKIVTGVPEAIPVIGSPLVELLRGSASVGQSTLTRFYSLHTFVLPLLTAVFMLMHFPMIRKQGISGPL



#### __rpl16__

In [32]:
name = "rpl16"
off = -8
ref = arabidopsis
for ft in ref.features:
    if ft.type == "CDS":
        if ft.qualifiers["gene"][0] == name:
            target = ft.extract(arabidopsis).seq.translate(table=11, cds=True)

query = alt_loc[name][off].extract(sacha).seq.translate(table=11, cds=True)
alns = aligner.align(target, query)

In [33]:
print(alns[0])

M-LSPKRTRFRKQHRGRLKGISSRGNRICFGRYALQTLEPAWITSRQIEAGRRAMTRNVRRGGKIWVRIFPDKPVTVRPAETRMGSGKGSPEYWVAVVKPGKILYEMGGVPENIARKAISIAASKMPIKTQFIISE
|-..|||||||||||||.|||..|||||||||||||.|||||||||||||||||||||.|||||||||||||||||.||.||||||||||||||||.||||.||||||||.|||||.|||||||||||.||||||.
MNYNPKRTRFRKQHRGRMKGIAFRGNRICFGRYALQALEPAWITSRQIEAGRRAMTRNARRGGKIWVRIFPDKPVTLRPTETRMGSGKGSPEYWVAIVKPGRILYEMGGVSENIARRAISIAASKMPIRTQFIISG



#### __petD__

In the case of petD that has 2 options, these were compared first between them to choose one.

In [34]:
name = "petD"
off = -25
ref = arabidopsis
for ft in ref.features:
    if ft.type == "CDS":
        if ft.qualifiers["gene"][0] == name:
            target = ft.extract(arabidopsis).seq.translate(table=11, cds=True)

query = alt_loc[name][off].extract(sacha).seq.translate(table=11, cds=True)
alns = aligner.align(target, query)

In [35]:
print(alns[0])

M------GVTKKPDLNDPVLRAKLAKGMGHNYYGEPAWPNDLLYIFPVVILGTIACNVGLAVLEPSMIGEPADPFATPLEILPEWYFFPVFQILRTVPNKLLGVLLMVSVPAGLLTVPFLENVNKFQNPFRRPVATTVFLIGTAAALWLGIGATLPIDKSLTLGLF
|------..|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||.|||||||||||||||||||||
MYKNSPIQITKKPDLNDPVLRAKLAKGMGHNYYGEPAWPNDLLYIFPVVILGTIACNVGLAVLEPSMIGEPADPFATPLEILPEWYFFPVFQILRTVPNKLLGVLLMVSVPAGLLTVPFLENVNKFQNPFRRPVATTVFLIGTAVALWLGIGATLPIDKSLTLGLF



In [36]:
name = "petD"
off = -49
ref = arabidopsis
for ft in ref.features:
    if ft.type == "CDS":
        if ft.qualifiers["gene"][0] == name:
            target = ft.extract(arabidopsis).seq.translate(table=11, cds=True)

query = alt_loc[name][off].extract(sacha).seq.translate(table=11, cds=True)
alns = aligner.align(target, query)

In [37]:
print(alns[0])

M-----G---------VTKKPDLNDPVLRAKLAKGMGHNYYGEPAWPNDLLYIFPVVILGTIACNVGLAVLEPSMIGEPADPFATPLEILPEWYFFPVFQILRTVPNKLLGVLLMVSVPAGLLTVPFLENVNKFQNPFRRPVATTVFLIGTAAALWLGIGATLPIDKSLTLGLF
|-----|---------.|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||.|||||||||||||||||||||
MSGSFGGWIYKNSPIQITKKPDLNDPVLRAKLAKGMGHNYYGEPAWPNDLLYIFPVVILGTIACNVGLAVLEPSMIGEPADPFATPLEILPEWYFFPVFQILRTVPNKLLGVLLMVSVPAGLLTVPFLENVNKFQNPFRRPVATTVFLIGTAVALWLGIGATLPIDKSLTLGLF



As this case was similar to the petB case, the same strategy was used: compare to a closer reference (_Ricinus communis_)

In [38]:
name = "petD"
off = -49
ref = ricinus
for ft in ref.features:
    if ft.type == "CDS":
        if ft.qualifiers["gene"][0] == name:
            target = ft.extract(ref).seq.translate(table=11)

query = alt_loc[name][off].extract(sacha).seq.translate(table=11)
alns = aligner.align(target, query)

In [39]:
print(alns[0])

MSGSFGGWIYKNSPIPITKKPDLNDPVLRAKLAKGMGHNYYGEPAWPNDLLYIFPVVILGTIACNVGLAVLEPSMIGEPADPFATPLEILPEWYFFPVFQILRTVPNKLLGVLLMVSVPAGLLTVPFLENVNKFQNPFRRPVATTVFLVGTAVALWLGIGATLPIDKSLTLGLF*
|||||||||||||||.||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||.||||||||||||||||||||||||||
MSGSFGGWIYKNSPIQITKKPDLNDPVLRAKLAKGMGHNYYGEPAWPNDLLYIFPVVILGTIACNVGLAVLEPSMIGEPADPFATPLEILPEWYFFPVFQILRTVPNKLLGVLLMVSVPAGLLTVPFLENVNKFQNPFRRPVATTVFLIGTAVALWLGIGATLPIDKSLTLGLF*



As the -49 offset matched almost perfectly,  it is the chosen candidate. 

### __Remaining CDS__

After the 5 cds that have abnormal starting sites were adressed, there were still 9 cds to review with excess or stop codon problems. 

The cds that remained which ones do remain?

In [40]:
rem_ft = {ft.qualifiers["gene"][0] for ft in anom_ft} - anom_start_ft

In [41]:
for key, value in anomalus.items():
    if key in rem_ft:
        print(key,value)

rpoC1 {'start': True, 'cod': 1, 'end': 0, 'index': 62}
atpF {'start': True, 'cod': 2, 'end': 13, 'index': 75}
ndhK {'start': True, 'cod': 2, 'end': 0, 'index': 99}
rpl22 {'start': True, 'cod': 1, 'end': 5, 'index': 188}
ycf1 {'start': True, 'cod': 1, 'end': False, 'index': 243}
ndhI {'start': True, 'cod': 0, 'end': 0, 'index': 254}
ndhG {'start': True, 'cod': 0, 'end': 2, 'index': 256}
ccsA {'start': True, 'cod': 2, 'end': 0, 'index': 264}
ndhF {'start': True, 'cod': 1, 'end': 0, 'index': 270}
ycf1-fragment {'start': True, 'cod': 2, 'end': 0, 'index': 272}


Similarly as the previous cds, modified proteins were proposed after searching for missing stop codons,  but no offset was needed.

In [42]:
for ft in anom_ft:
    name = ft.qualifiers["gene"][0]
    if name in rem_ft:
        print(f"Beginnig with {name}")
        alt_loc[name] = {}
        alt_loc[name][0] = find_stop(ft, sacha, 11, offset = 0)

Beginnig with rpoC1
A stop for rpoC1 was found 3 codons downstream (9 bp) and the protein length would be 683 aa
Beginnig with atpF
This feature already has 13 stop codons
And the first of them is located at position 48 of 184
Beginnig with ndhK
A stop for ndhK was found 1 codons downstream (3 bp) and the protein length would be 226 aa
Beginnig with rpl22
This feature already has 5 stop codons
And the first of them is located at position 145 of 185
Beginnig with ycf1
This feature already has 1 stop codons
And it is located at position 1897 of 1904
Beginnig with ndhI
A stop for ndhI was found 7 codons downstream (21 bp) and the protein length would be 172 aa
Beginnig with ndhG
This feature already has 2 stop codons
And the first of them is located at position 176 of 178
Beginnig with ccsA
A stop for ccsA was found 2 codons downstream (6 bp) and the protein length would be 322 aa
Beginnig with ndhF
A stop for ndhF was found 1 codons downstream (3 bp) and the protein length would be 752 a

Although stop codons were found for some of the remaining proteins (rpoC1, ndhK, ndhI, ccsA, ndhF), others need further assessment such as atpF, rpl22, ycf1, ndhG as they have multiple stop codons.

The alignments for those cds that were fixed:

In [43]:
for name, dict_loc in alt_loc.items():
    if 0 in dict_loc.keys() and dict_loc[0]:
        if name == "ycf1-fragment": 
            continue
        else:
            query = dict_loc[0].extract(sacha).seq.translate(table=11, cds=True)

            for ft in arabidopsis.features:
                if ft.type == "CDS" and ft.qualifiers["gene"][0] == name:
                    target = ft.extract(arabidopsis).seq.translate(table=11, cds=True)

            alns = aligner.align(target, query)
            print(f"Alignment for {name} query {len(query)} target {len(target)}")
            print(alns[0])
        

Alignment for rpoC1 query 682 target 680
MIDRYKHQQLRIGLVSPQQISAWATKIIPNGEIVGEVTKPYTFHYKTNKPEKDGLFCERIFGPIKSGICACGNYRVIGDEKEDPKFCEQCGVEFVDSRIRRYQMGYIKLTCPVTHVWYLKRLPSYIANLLDKPLKELEGLVYCDFSFARPITKKPTFLRLRGSFEYEIQSWKYSIPLFFTTQGFDIFRNREISTGAGAIREQLADLDLRIIIENSLVEWKQLGEEGPTGNEWEDRKIVRRKDFLVRRMELAKHFIRTNIEPEWMVLCLLPVLPPELRPIIQIEGGKLMSSDINELYRRVIYRNNTLTDLLTTSRSTPGELVMCQEKLVQEAVDTLLDNGIRGQPMRDGHNKVYKSFSDVIEGKEGRFRETLLGKRVDYSGRSVIVVGPSLSLHRCGLPREIAIELFQTFVIRGLIRQHLASNIGVAKSQIREKKPIVWEILQEVMQGHPVLLNRAPTLHRLGIQSFQPILVEGRTICLHPLVCKGFNADFDGDQMAVHVPLSLEAQAEARLLMFSHMNLLSPAIGDPISVPTQDMLIGLYVLTSGTRRGICANRYNPCNRKNYQNERIYETNYKYTKEPFFCNSYDAIGAYRQKKINLDSPLWLRWQLDQRVIASREVPIEVHYESFGNYHEIYAHYLIVRSVKKENFCIYIRTTVGHISFYREIEEAIQGFSQACS--YDT
||||||||||||||||||||||||.||.|||||||||||||||||||||||||||||||||||||||||||||||||..|||.|||||||||||||||.||||||||||.|||||||||||||||||||||||||||||||||||||||||.||||||||||||||||||||||||||||.||||.||||||||||||||.|||||||||||..|.||||.|||.|||||||||||..|||||||||.||||||||||||||||||||||||||||

All the alternatives seem to be appropiate.

Focusing again on the ones with multiple stop codons they were adressed individually. As they have multiple stop codons, it was likely that the reading frame was not the appropiate. Hence translation and alignment in different reading frames was tested for these cds

#### __atpF__

In [44]:
idx = anomalus["atpF"]["index"]
ft = sacha.features[idx]
name = ft.qualifiers["gene"][0]

for i in range(3):
    query = (ft.location + -i).extract(sacha).seq.translate(table=11)
    
    for ft in arabidopsis.features:
        if ft.type == "CDS" and ft.qualifiers["gene"][0] == name:
                target = ft.extract(arabidopsis).seq.translate(table=11)
                
    alns = aligner.align(target,query)
    print(f"Alignment with offset {-1*i} and score {alns.score}")
    print(alns[0])

Alignment with offset 0 and score 246.5
MKNLTDSFVYLGHWPSAGSFGFNTDILATNPINLSVVFGVLIFFGKGV--LNDLLDNRKQRIL-----NTIR---NSEELREG-AI-----QQL--E-------NAR--------ARLRN---VETEADKFRVNGYSEIEREKL----N-LINSTYKTLKQL------ENYKNETILFEQQRTINQVRERVFQQALQGAIGTLNSCLSNELHLRTINANIGMFGTMKE--ITD*
|||.|||||.|||||||.|||||||||||||||||||.||||||||||--......|....||-----|..|---||..-..|-|.-----.|.--|-------|.|--------.||.|---.-|....|..|-----..|.|----|-..|..|..|...------..|-------------------......-----....||----------------|..|.--|---
MKNVTDSFVSLGHWPSAESFGFNTDILATNPINLSVVLGVLIFFGKGV*LIY*IIENKRFWILFEIQKNYARGPLNSWK-KPGPAYGKWK*KQISFE*MDTRR*NERN*I*LIQLIRLWNN*KI-TKMKPFILN-----NKERLIKSDNGFSNKPYREL*EF*IVV*PTSY-------------------IYVPSM-----LILACL----------------GR*KK*LI---

Alignment with offset -1 and score 65.0
MKNLTDSFVYLGHWP-SAGSFGFNTDILATNPINLSVVFGVLI-FFGKGVLNDL-------------LD--------NRKQRIL--------------------NTIRNSEEL------REGAIQQLENARARLRNVET---------EA---DKF----RVNG-YSEI----E--------REKL--NLINSTYKTL---------KQ-----

Although the first alignment had the highest score, it was not considerably higher than the other ones. Furthermore, that same alignment had a high similarity at the start of the sequence which seemed different from the middle and the end of the sequence. Further inspection revealed that this cds was composed by a short exon and other longer.

In [45]:
idx = anomalus["atpF"]["index"]
list(map(len,sacha.features[idx].location.parts))

[144, 410]

Evaluating the reading frames for the exons individually:

In [46]:
idx = anomalus["atpF"]["index"]
ex1, ex2 = sacha.features[idx].location.parts

print("exon 1")
for i in range(0,-3,-1):
    count = (ex1 + i).extract(sacha).seq.translate(table=11).count("*")
    print(f"Has {count} stop codons in reading frame {i}")
    
print("exon 2")
for i in range(0,-3,-1):
    count = (ex2 + i).extract(sacha).seq.translate(table=11).count("*")
    print(f"Has {count} stop codons in reading frame {i}")

exon 1
Has 0 stop codons in reading frame 0
Has 1 stop codons in reading frame -1
Has 7 stop codons in reading frame -2
exon 2
Has 13 stop codons in reading frame 0
Has 0 stop codons in reading frame -1
Has 10 stop codons in reading frame -2


The second exon was not reading the appropiate frame, although it still misses the stop codon. 

In [47]:
idx = anomalus["atpF"]["index"]
name = sacha.features[idx].qualifiers["gene"][0]
exon = sacha.features[idx].location.parts[-1] + -1
query = (sacha.features[idx].location.parts[0] + exon).extract(sacha).seq.translate(table=11)

for ft in arabidopsis.features:
    if ft.type == "CDS" and ft.qualifiers["gene"][0] == name:
        target = ft.extract(arabidopsis).seq.translate(table=11)

alns = aligner.align(target,query)
print(f"Alignment with offset -{1} in the last exon and score {alns.score}")
print(alns[0])

Alignment with offset -1 in the last exon and score 820.0
MKNLTDSFVYLGHWPSAGSFGFNTDILATNPINLSVVFGVLIFFGKGVLNDLLDNRKQRILNTIRNSEELREGAIQQLENARARLRNVETEADKFRVNGYSEIEREKLNLINSTYKTLKQLENYKNETILFEQQRTINQVRERVFQQALQGAIGTLNSCLSNELHLRTINANIGMFGTMKEITD*
|||.|||||.|||||||.|||||||||||||||||||.|||||||||||.||||||||.||.|||||||||||||.|||.||||||.||.|||.||.|||||||||||||||||||||.||||||||||.|||||||||||.||||||||||.|.|||||.|||||||||||.|||||.||||.-
MKNVTDSFVSLGHWPSAESFGFNTDILATNPINLSVVLGVLIFFGKGVLTDLLDNRKQKILDTIRNSEELREGAIEQLEKARARLRKVEIEADQFRMNGYSEIEREKLNLINSTYKTLEQLENYKNETIHFEQQRTINQVRQRVFQQALQGALGILNSCLTNELHLRTINANLGMFGTIKEITY-



It just misses the stop codon:

In [48]:
idx = anomalus["atpF"]["index"]
exon = sacha.features[idx].location.parts[-1] 
loc = (sacha.features[idx].location.parts[0] + FeatureLocation(exon.start + -1, exon.end, exon.strand))
query = loc.extract(sacha).seq.translate(table=11)

for ft in arabidopsis.features:
    if ft.type == "CDS" and ft.qualifiers["gene"][0] == name:
        target = ft.extract(arabidopsis).seq.translate(table=11)
        
alns = aligner.align(target,query)
print(f"Alignment score {alns.score}")
print(alns[0])

alt_loc["atpF"] = {0:loc}

Alignment score 826.0
MKNLTDSFVYLGHWPSAGSFGFNTDILATNPINLSVVFGVLIFFGKGVLNDLLDNRKQRILNTIRNSEELREGAIQQLENARARLRNVETEADKFRVNGYSEIEREKLNLINSTYKTLKQLENYKNETILFEQQRTINQVRERVFQQALQGAIGTLNSCLSNELHLRTINANIGMFGTMKEITD*
|||.|||||.|||||||.|||||||||||||||||||.|||||||||||.||||||||.||.|||||||||||||.|||.||||||.||.|||.||.|||||||||||||||||||||.||||||||||.|||||||||||.||||||||||.|.|||||.|||||||||||.|||||.||||.|
MKNVTDSFVSLGHWPSAESFGFNTDILATNPINLSVVLGVLIFFGKGVLTDLLDNRKQKILDTIRNSEELREGAIEQLEKARARLRKVEIEADQFRMNGYSEIEREKLNLINSTYKTLEQLENYKNETIHFEQQRTINQVRQRVFQQALQGALGILNSCLTNELHLRTINANLGMFGTIKEITY*



#### __rpl22__

As rpl22 had several stop codons, its reading frame was checked first:

In [49]:
idx = anomalus["rpl22"]["index"]
aux = sacha.features[idx].location

for i in range(0,-3,-1):
    prot = (aux + i).extract(sacha).seq.translate(table=11)
    count = prot.count("*")
    print(f"Has {count} stop codons in reading frame {i}") 
    print(f"the first in {prot.find('*')} of {len(prot)}")

Has 5 stop codons in reading frame 0
the first in 145 of 185
Has 20 stop codons in reading frame -1
the first in 0 of 185
Has 9 stop codons in reading frame -2
the first in 2 of 185


The reading frame was the correct one. However, If the start codon and the reading frame were already set, the only thing that could be done in this case was to set the end of the cds where the first stop codon is found 

In [50]:
def early_stop(feature, mother, gen_cod):
    """Given a feature and its mother sequence that has an early
    stop codon, it returns a modified location with the corrected
    end at the first stop codon"""
    excess = len(feature.location) % 3
    orig_len = len(feature.location.extract(mother).seq.translate(table=gen_cod))
    stop_len = len(feature.location.extract(mother).seq.translate(table=gen_cod, to_stop = True))
    mod = (orig_len - stop_len - 1)*3 + excess
    loc = feature.location
    if loc.strand == -1:
        mod_loc = FeatureLocation(loc.start - mod*loc.strand , loc.end, loc.strand)
    elif loc.strand == 1:
        mod_loc = FeatureLocation(loc.start, loc.end - mod*loc.strand, loc.strand)

    #print(mod_loc.extract(sacha).seq.translate(gen_cod, cds=True))
    return mod_loc

In [51]:
idx = anomalus["rpl22"]["index"]
name = sacha.features[idx].qualifiers["gene"][0]

query = early_stop(sacha.features[idx], sacha, 11).extract(sacha).seq.translate(table=11, cds=True)
#query = sacha.features[idx].extract(sacha).seq.translate(table=11)

for ft in ricinus.features:
    if ft.type == "CDS" and ft.qualifiers["gene"][0] == name:
        target = ft.extract(ricinus).seq.translate(table=11, cds=True)

alns = aligner.align(target,query)
print(f"Alignment and score {alns.score} - target ({len(target)}) - query ({len(query)})")
print(alns[0])

Alignment and score 607.5 - target (160) - query (145)
MINKRKK-----KR---DPYTEVYALGQHICMSTHKARRIIDQIRGRSYEETLMILELMPYRACYPILKLIYSAAANARHNMGFNEANLIISKAEVNEGATVKKLKPQARGRGYLIKRSTCHITIVLKNISLYEEYEEYEEYNICLKNRGWIKKTKSTDATFHDMYSK
|||||||-----|.---||||.||||||||.||||||||||.||||||||||||||||||||||||||||||||||||||||||||..|||||||||||||||||||||||||||||||||||||||||||||||...|..|.|-|----------------------
MINKRKKGDPYTKEYALDPYTTVYALGQHISMSTHKARRIIEQIRGRSYEETLMILELMPYRACYPILKLIYSAAANARHNMGFNETDLIISKAEVNEGATVKKLKPQARGRGYLIKRSTCHITIVLKNISLYEESDGYSKYSI-L----------------------



In [52]:
alt_loc["rpl22"] = {0:early_stop(sacha.features[idx],sacha, 11)}

Given the early stop, further assessment would be required to be sure if the cds is functional.

#### __ycf1__

In [53]:
idx = anomalus["ycf1"]["index"]
name = sacha.features[idx].qualifiers["gene"][0]

query = sacha.features[idx].extract(sacha).seq.translate(table=11)

for ft in ricinus.features:
    if ft.type == "CDS" and ft.qualifiers["gene"][0] == name:
        target = ft.extract(ricinus).seq.translate(table=11)

alns = aligner.align(target,query)
print(f"Alignment and score {alns.score} - target ({len(target)}) - query ({len(query)})")
print(alns[0])

Alignment and score 7991.0 - target (1806) - query (1904)
MIFKSFILGNLVSLCMKIINLVVVVGLYYGFLTTFSMGPSYLFLLRARVIEEGEEGTEKKVSATTGFITGQLMMFISIYYAPLHLALGRPHTITVLALPYLLFHFFWNNHKHFFDYGATTRNSMRNLSIQFVFLNNLIFQLFNHFILPSSMLVRLVNIYMFRCNNKMLFVTSSFVGWLIGHILFMKWVGLILVWIQQNNSIRSNVLFRSNKYLVSELRNSMARIFSILLFITCVYSLGRIPSPIFTKKLKETSETEEREESEEETDVEIETTSETKGTKQGSTEEDPSSSLFSEEKEDRDKIDETEEIQVNGKEKTKDEFHFHFNETCYKNRPLYETFYLDGNQENSKLEILIEKKKKNLFWFEKPLVTILFDSKRWNRPLRYIKNDQFENAVRNEMSQYFFYTCRSDGKERISFTYPPSLSTFFEMVQKKIFLFTTTTEKLSSDELYNHWNYKNEQKKKNLSNEFINRVQALDKGYLVLNTLEKRTRLCNDKTKKEYLPKIYDPLLSGSYRGRTKFFFSPSTLNINKTSIKNSTEMVWINKIHLILLISNYIEFEPKTDRKSFSTEIAYFLNLINEFAGKSTSSFNFKGLPLFPDYKEEKMDLELLQILKFVFDPVLANSKNKTIKNNSL----GIKEINKQVPHWSYKLIDDLEQQEGENEENMAEDYEIRSRKAKRVVIFTDNQENTATYNNTKDTTDFDQIDEVTLIRYSQQSDFRRDIIKGSTRAQRRKIAIFELFQANAHSPLFLDRIDKSLFFSFDIFELIKTMFINWMCKNAEFTISDYTYT--EKKTKESKKKDEDTREDDKREERTRIEIAEAWDSILFAQIIRGCILVTQSILRKYIILPLLIIAKNCIRILFFQIPEWSEDLKDWSREMHIKCTYNGVQLSEKEFPKNWLTDGIQIKILFPFRLKPWHRSKLKFPHKDPMKKKVQRNDFC

ycf1 presented a case similar to rpl22: Both the reading frame and the start codon were the correct and already had an interior stop codon. Hence, the same procedure was used.

In [54]:
idx = anomalus["ycf1"]["index"]
alt_loc["ycf1"] = {0:early_stop(sacha.features[idx],sacha, 11)}

#### __ndhG__

Finally ndhG

In [55]:
idx = anomalus["ndhG"]["index"]
name = sacha.features[idx].qualifiers["gene"][0]
query = sacha.features[idx].extract(sacha).seq.translate(table=11)

for ft in arabidopsis.features:
    if ft.type == "CDS" and ft.qualifiers["gene"][0] == name:
        target = ft.extract(arabidopsis).seq.translate(table=11)

alns = aligner.align(target,query)
print(f"Alignment and score {alns.score}")
print(alns[0])

Alignment and score 739.0
MDLPGPIHDFLLVFLGSGLLVGGLGVVLLPNPIFSAFSLGFVLVCISLLYILSNSHFVAAAQLLIYVGAINVLIIFAVMFMNDSEYSTDFNLWTIGNGITSLVCTTILFLLMSTILDTSWYGVIWTTKLNQILEQDLISNSQQIGIHLSTDFFLPFELISIILLVALIGAISVARQ-*
||||..||||||.|||..|..||||||||.|||.||||||.|||||||.|||||||||||||||||||||||||||||||||.|||...|||||.|||.||||||.|...|...||||||||.|||||.|||||||||||.||||||||||||||||.|||||||||||||.||||-|
MDLPALIHDFLLFFLGLVLILGGLGVVLLTNPIYSAFSLGLVLVCISLFYILSNSHFVAAAQLLIYVGAINVLIIFAVMFMNGSEYYKEFNLWTVGNGVTSLVCTSIFVSLITIILDTSWYGIIWTTKTNQILEQDLISNGQQIGIHLSTDFFLPFEFISIILLVALIGAIAVARQ**



ndhG had a duplicated stop codon as their only fault.

In [56]:
alt_loc["ndhG"] = {0:early_stop(sacha.features[291], sacha, 11)}

## __Wrap up__

To summarize:

In [57]:
anomalus

{'rpoC1': {'start': True, 'cod': 1, 'end': 0, 'index': 62},
 'atpF': {'start': True, 'cod': 2, 'end': 13, 'index': 75},
 'ndhK': {'start': True, 'cod': 2, 'end': 0, 'index': 99},
 'petB': {'start': False, 'cod': 1, 'end': 13, 'index': 168},
 'petD': {'start': False, 'cod': 2, 'end': 18, 'index': 170},
 'infA': {'start': False, 'cod': 0, 'end': 2, 'index': 178},
 'rpl16': {'start': False, 'cod': 1, 'end': 10, 'index': 184},
 'rpl22': {'start': True, 'cod': 1, 'end': 5, 'index': 188},
 'ycf1': {'start': True, 'cod': 1, 'end': False, 'index': 243},
 'ndhI': {'start': True, 'cod': 0, 'end': 0, 'index': 254},
 'ndhG': {'start': True, 'cod': 0, 'end': 2, 'index': 256},
 'ndhD': {'start': False, 'cod': 0, 'end': True, 'index': 262},
 'ccsA': {'start': True, 'cod': 2, 'end': 0, 'index': 264},
 'ndhF': {'start': True, 'cod': 1, 'end': 0, 'index': 270},
 'ycf1-fragment': {'start': True, 'cod': 2, 'end': 0, 'index': 272}}

In [58]:
alt_loc

{'petB': {37: None,
  46: None,
  -41: FeatureLocation(ExactPosition(81013), ExactPosition(81700), strand=1)},
 'petD': {17: None,
  44: None,
  56: None,
  -1: None,
  -7: None,
  -25: FeatureLocation(ExactPosition(82687), ExactPosition(83188), strand=1),
  -49: FeatureLocation(ExactPosition(82663), ExactPosition(83188), strand=1)},
 'rpl16': {43: None,
  52: None,
  -8: FeatureLocation(ExactPosition(86871), ExactPosition(87282), strand=-1),
  -38: None},
 'rpoC1': {0: CompoundLocation([FeatureLocation(ExactPosition(31820), ExactPosition(32252), strand=1), FeatureLocation(ExactPosition(33058), ExactPosition(34675), strand=1)], 'join')},
 'atpF': {0: CompoundLocation([FeatureLocation(ExactPosition(43236), ExactPosition(43380), strand=1), FeatureLocation(ExactPosition(44098), ExactPosition(44509), strand=1)], 'join')},
 'ndhK': {0: FeatureLocation(ExactPosition(53113), ExactPosition(53791), strand=-1)},
 'rpl22': {0: FeatureLocation(ExactPosition(89420), ExactPosition(89858), strand=-1)

There were 15 anomalus cds from which we curated and have new locations for 13 of them. The remaining 2 were ycf1-fragment and infA; the former being a natural fragmentation and the later a suspect for sporious match.

Finally, to make a final genbank file, the cds had to be replaced and infA should be removed. It was also noted that the gene pbf1 is named as psbN and other chloroplast; so it is sensible to rename it to preserve the convention. Additionally, in reference files the cds usually include the product of the annotation. Furthermore, as a result of a heteroplasmy search on the chlroplast, it was found that the only ambiguous nucleotide in the sequence (W at 17517), has enough depth to be disambiguated (it is an A).

But first, the dictionary with the modified locations had to be organized:

In [59]:
del alt_loc["ycf1-fragment"]
for key in alt_loc.keys():
    alt_loc[key] = {key2:value for key2, value in alt_loc[key].items() if value != None}

In [60]:
alt_loc

{'petB': {-41: FeatureLocation(ExactPosition(81013), ExactPosition(81700), strand=1)},
 'petD': {-25: FeatureLocation(ExactPosition(82687), ExactPosition(83188), strand=1),
  -49: FeatureLocation(ExactPosition(82663), ExactPosition(83188), strand=1)},
 'rpl16': {-8: FeatureLocation(ExactPosition(86871), ExactPosition(87282), strand=-1)},
 'rpoC1': {0: CompoundLocation([FeatureLocation(ExactPosition(31820), ExactPosition(32252), strand=1), FeatureLocation(ExactPosition(33058), ExactPosition(34675), strand=1)], 'join')},
 'atpF': {0: CompoundLocation([FeatureLocation(ExactPosition(43236), ExactPosition(43380), strand=1), FeatureLocation(ExactPosition(44098), ExactPosition(44509), strand=1)], 'join')},
 'ndhK': {0: FeatureLocation(ExactPosition(53113), ExactPosition(53791), strand=-1)},
 'rpl22': {0: FeatureLocation(ExactPosition(89420), ExactPosition(89858), strand=-1)},
 'ycf1': {0: FeatureLocation(ExactPosition(116127), ExactPosition(121821), strand=1)},
 'ndhI': {0: FeatureLocation(Ex

Thus

In [61]:
new_sacha_fts = sacha.features.copy()

for name, dict_loc in alt_loc.items():
    idx = anomalus[name]["index"] - 1
    for offset, loc in dict_loc.items():
        while new_sacha_fts[idx].qualifiers["gene"][0] == name:
            if new_sacha_fts[idx].type in {"gene", "CDS"}:
                new_sacha_fts[idx].location = loc
            elif new_sacha_fts[idx].type == "exon":
                num = int(new_sacha_fts[idx].qualifiers["number"][0])
                new_sacha_fts[idx].location = loc.parts[num - 1]
            elif new_sacha_fts[idx].type == "intron":
                num = int(new_sacha_fts[idx].qualifiers["number"][0])
                start = loc.parts[num -1].end
                end = loc.parts[num].start
                new_sacha_fts[idx].location = FeatureLocation(start, end, loc.strand)
            if "gene" not in new_sacha_fts[idx + 1].qualifiers.keys():
                idx += 2 ## This was cheating, it works just for this case. Not aimed for general use
            idx += 1

To filter the infA and rename "pbf1" to "psbN":

In [62]:
new_sacha_fts_filtered = []
for ft in new_sacha_fts:
    if "gene" in ft.qualifiers.keys():
        if ft.qualifiers["gene"][0] != "infA":
            new_sacha_fts_filtered.append(ft)
            if ft.qualifiers["gene"][0] == "pbf1":
                ft.qualifiers["gene"][0] = "psbN"
    else:
        new_sacha_fts_filtered.append(ft)
        

Now to add the translation products to the cds and other qualifiers to be NCBI compliant (add product, translation table) :

In [63]:
definite_features = []
for i,ft in enumerate(new_sacha_fts_filtered):
    if ft.type == "CDS":
        name = ft.qualifiers["gene"][0]
        if name == "ndhD": ## Remember ndhD has a particular starting site
            protein = "M" + ft.extract(sacha).seq.translate(table=11)[1:-1]
            ft.qualifiers["transl_except"] = ["(pos:1..3,aa:Met)"]
        #elif name == "ycf1-fragment":
        #    continue
        else:
            try:
                protein = ft.extract(sacha).seq.translate(table=11, cds=True)
            except:
                print(i, name)
                ft.qualifiers["exception"] = "Opposite IR fragment of protein ycf1"
                protein = ft.extract(sacha).seq.translate(table=11, to_stop=True)
                
        ## Look for the description in ricinus chloroplast
        for feat in ricinus.features:
            if feat.type == "CDS" and feat.qualifiers["gene"][0] == name:
                prod = feat.qualifiers["gene"][0]
        
        ft.qualifiers["product"] = [prod]
        ft.qualifiers["translation"] = [str(protein)]
        ft.qualifiers["transl_table"] = ["11"]
        definite_features.append(ft)
    else:
        definite_features.append(ft)

270 ycf1-fragment


Finally the genbank file is created

In [64]:
gb = SeqIO.read("chloroplast_annotated_raw.gb", "gb")
gb.features = definite_features
gb.seq = Seq(str(gb.seq).replace("W","A"), gb.seq.alphabet)
SeqIO.write(gb, "chloroplast_annotated_curated.gb", "gb")

1